# Fatal Police Shooting in the United States

## Author: Martynas Loveikis

**Welcome to this notebook 3 part series, where we bravely venture into the statistical minefield of the Fatal Police Shootings in the US dataset, courtesy of The Washington Post.**

### **Objective: Cops, Mental Health or the Lunar Cycle -  Is There a Connection?**

The moon's influence is a tale as old as time. But does it extend to those high-pressure moments on the beat?  We're analyzing if mental health-related incidents align with the lunar phases. 

In this oh-so-delicate analysis, we'll sanitize the data, engineer suspiciously relevant new features, and visualize the heck out of it. Our goal? To glean insights into America's **contentious** fascination with **police brutality** over the past decade. Hold onto your hats, folks. 

### 1. **Initialization**

_We begin by importing the necessary libraries and loading the dataset._

In [1]:
import ephem
import holidays
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm_notebook

In [2]:
shootings_url = ('https://raw.githubusercontent.com/washingtonpost/'
                 'data-police-shootings/master/v2/'
                 'fatal-police-shootings-data.csv')
df = pd.read_csv(shootings_url, index_col=0).drop('agency_ids', axis=1)

In [3]:
df.head()

,date,threat_type,flee_status,armed_with,city,county,state,latitude,longitude,location_precision,name,age,gender,race,race_source,was_mental_illness_related,body_camera
id,,,,,,,,,,,,,,,,,
3,2015-01-02,point,not,gun,Shelton,Mason,WA,47.246826,-123.121592,not_available,Tim Elliot,53.0,male,A,not_available,True,False
4,2015-01-02,point,not,gun,Aloha,Washington,OR,45.487421,-122.891696,not_available,Lewis Lee Lembke,47.0,male,W,not_available,False,False
5,2015-01-03,move,not,unarmed,Wichita,Sedgwick,KS,37.694766,-97.280554,not_available,John Paul Quintero,23.0,male,H,not_available,False,False
8,2015-01-04,point,not,replica,San Francisco,San Francisco,CA,37.762910,-122.422001,not_available,Matthew Hoffman,32.0,male,W,not_available,True,False
9,2015-01-04,point,not,other,Evans,Weld,CO,40.383937,-104.692261,not_available,Michael Rodriguez,39.0,male,H,not_available,False,False


In [4]:
print(f'First incident:	{df['date'].min()}')
print(f'Last incident:	{df['date'].max()}')

First incident:	2015-01-02
Last incident:	2024-03-26


_**Note:** The dataset covers slightly less than a full decade. This could be a factor to consider when interpreting trends.  Sadly, even with this incomplete picture, we're already approaching a staggering 10,000 counts of police brutality. That translates to roughly a thousand lives lost each year, a stark reminder of the issue's gravity._ 


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9497 entries, 3 to 10272
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   date                        9497 non-null   object 
 1   threat_type                 9430 non-null   object 
 2   flee_status                 8199 non-null   object 
 3   armed_with                  9285 non-null   object 
 4   city                        9427 non-null   object 
 5   county                      4775 non-null   object 
 6   state                       9497 non-null   object 
 7   latitude                    8446 non-null   float64
 8   longitude                   8446 non-null   float64
 9   location_precision          8446 non-null   object 
 10  name                        9152 non-null   object 
 11  age                         9110 non-null   float64
 12  gender                      9470 non-null   object 
 13  race                        8372 non-

In [6]:
print(f"The number of nulls in each feature is: \n{df.isnull().sum()}")

The number of nulls in each feature is: 
date                             0
threat_type                     67
flee_status                   1298
armed_with                     212
city                            70
county                        4722
state                            0
latitude                      1051
longitude                     1051
location_precision            1051
name                           345
age                            387
gender                          27
race                          1125
race_source                   1098
was_mental_illness_related       0
body_camera                      0
dtype: int64


### 2. **The Geocoding Quest: Restoring Location Order** 

_Someone clearly slept through geography class.  Let's see if a reverse geocoding API can fix this mess._

In [7]:
def get_missing_coordinates(dataframe, location_column):
	"""Identifies rows with missing location data but valid coordinates.
    Args:
        dataframe (pandas.DataFrame): Input DataFrame.
        location_column (str): Name of the location column.
    Returns:
        pandas.DataFrame: DataFrame containing rows with 
        missing location data but valid coordinates.
    """

	return dataframe.loc[dataframe[location_column].isna() &
	                     ~dataframe['latitude'].isna() &
	                     ~dataframe['longitude'].isna(),
	['latitude', 'longitude']]


def extract_available_values(dataframe, location_column):
	"""Identifies the values that were available in the given coordinates.
    Args:
        dataframe (pandas.DataFrame): Input DataFrame.
        location_column (str): Name of the location column.
    Returns:
        pandas.DataFrame: DataFrame containing available values.
    """

	return ~dataframe.loc[dataframe[location_column].isna(),
	['latitude', 'longitude', location_column]]


def reverse_geocode(latitude, longitude, location_type='city'):
	"""Retrieves location information (city or county) from coordinates.
    Args:
        latitude (float): Latitude coordinate.
        longitude (float): Longitude coordinate.
        location_type (str, optional): Type of location to retrieve.
        Defaults to 'city'.
    Returns:
        str: Location information.
    """

	geolocator = Nominatim(user_agent='reverse_geocoding')
	location = geolocator.reverse((latitude, longitude), exactly_one=True)
	if location:
		address = location.raw['address']
		return address.get(location_type, 'not_available')
	else:
		return 'not_available'


def get_location_from_coords(row, location_type):
	"""Retrieves location information from coordinates.
    Args:
        row (pandas.Series): Row containing latitude and longitude.
        location_type (str): Type of location to retrieve.
    Returns:
        str: Location information.
    """

	return reverse_geocode(row['latitude'], row['longitude'], location_type)


def fill_geo_locations(dataframe, location_type):
	"""Fills missing location data using reverse geocoding.
    Args:
        dataframe (pandas.DataFrame): DataFrame with 'latitude', 'longitude', 
        and missing location columns.
        location_type (str): 'city' or 'county'.
    """

	missing_locations = get_missing_coordinates(dataframe, location_type)
	tqdm_notebook.pandas()

	# Use progress_apply instead of apply
	missing_locations[location_type] = missing_locations.progress_apply(
		lambda row: get_location_from_coords(row, location_type), axis=1)
	updated_locations = extract_available_values(missing_locations, location_type)
	dataframe[location_type] = (
		dataframe[location_type].combine_first(updated_locations[location_type]))


fill_geo_locations(df, 'city')
fill_geo_locations(df, 'county')

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/3917 [00:00<?, ?it/s]

### Data Cleanup & Simplification

Time to roll up our sleeves and tackle this messy dataset.  Our strategy:

* **Missing Ages:** Impute with the median age – at least they'll be statistically average.
* **Missing Coordinates:**  Replace with (0, 0). Might as well locate these incidents in the middle of the ocean.
* **Other Missing Features:**  Change to 'not_available' for consistency. 
* **Redundant Categories:** Consolidate similar values to 'not_available' – simplifies things for our weary brains.



In [8]:
numerical_columns = ['latitude', 'longitude', 'age']
age_median = df['age'].median()


def transform_column(column, df):
	"""
    Imputes missing values and creates indicator columns for substitutions.
    Handles numerical and non-numerical columns differently:
    * Numerical:
        - 'latitude', 'longitude': Filled with 0,
        creates 'substituted_location' indicator.
        - 'age': Filled with median, creates 'substituted_age' indicator.
    * Non-numerical:
        - Filled with 'not_available'
        - Replaces 'unknown' and 'undetermined' with 'not_available' 
    """

	if column in numerical_columns:
		if column in ['latitude', 'longitude']:
			df[column] = df[column].fillna(0)
			df.loc[df[column] == 0, 'substituted_location'] = True
		elif column == 'age':
			df[column] = df[column].fillna(age_median)
			df.loc[df[column] == age_median, 'substituted_age'] = True
	else:
		df[column] = df[column].fillna('not_available')
		df[column] = df[column].replace(
			{'Unknown'     : 'not_available',
			 'unknown'     : 'not_available',
			 'undetermined': 'not_available'})


df['substituted_location'] = False
df['substituted_age'] = False

for column in df.columns:
	transform_column(column, df)

### Improving Feature Clarity

Honestly, this feature is a bit of a mess – even I'm starting to doubt its original purpose. Let's break it down and make it more readable. 

In [9]:
df['race'] = df['race'].astype('string')


def decode_race(value):
	"""
    Decodes race abbreviations into full descriptions.
    Handles multiple race codes separated by semicolons.
    Replaces unknown codes with 'Unknown'.
    Args:
        value (str): A string containing race codes
        separated by semicolons (e.g., 'W;B').
    Returns:
        str: A string with decoded race descriptions
        separated by semicolons (e.g., 'White;Black').
    """

	race_mapping = {
		'W' : 'White',
		'B' : 'Black',
		'A' : 'Asian heritage',
		'N' : 'Native American',
		'H' : 'Hispanic',
		'O' : 'Other',
		'--': 'Unknown'
	}
	decoded_values = []
	for code in value.split(';'):
		decoded_values.append(race_mapping.get(code.strip(), 'Unknown'))
	return ';'.join(decoded_values)


df['race'] = df['race'].apply(decode_race)

### 3. **Feature Engineering Frenzy**

_Let's get creative and see what insights we can squeeze out of this dataset._

* **Brainstorm Time!** List out every potential feature you can think of. The wilder, the better (for now).  We'll evaluate them later.

In [10]:
city_counts = df['city'].value_counts()
county_counts = df['county'].value_counts()


def count_fatalities(row):
	"""
    Calculates the number of fatalities
    associated with a location in the DataFrame.
    Args:
        row (pandas.Series): A single row from the DataFrame containing 'city', 
                             'county', and 'fatalities' columns.
    Returns:
        int: The number of fatalities associated with the location (city or county).
    """

	if row['city'] == 'not_available':
		return city_counts.get(row['county'], 0)
	else:
		return county_counts.get(row['city'], 0)


df['location_fatality_counter'] = (
	df.apply(count_fatalities, axis=1)
	.astype(int))

### Categorizing Age for Impact

You're right – seeing **toddler** or **elderly** adds an emotional dimension that raw age numbers don't have. Let's create age categories to highlight these vulnerable groups.  


In [11]:
bins = [0, 3, 13, 20, 30, 60, 76, 90, 100]
labels = ['toddler', 'child', 'teenager', 'young adult',
          'adult', 'senior', 'elderly', 'very elderly']
df['age_group'] = (
	pd.cut(df['age'],
	       bins=bins, labels=labels)).astype('category')

### Calculating a Threat Level Score

Consolidating those three features makes sense. Let's create a composite **threat level** score that gives us an instant snapshot of the situation's intensity. We can even incorporate **weighted values** for a more nuanced assessment. 


In [12]:
df['armed_with'] = (
	df['armed_with'].astype(str))


def scale_armed_threat_score(threat_type_str):
	"""
    This function takes a string containing threat types (separated by ';')
    and returns the highest danger score
    based on the 'weapon_threat_ratio' dictionary.
    Args:
        threat_type_str: String containing threat types (e.g., 'gun;other')
    Returns:
        int: Highest danger score found in the string.
    """

	weapon_threat_ratio = {
		'gun'          : 1,
		'vehicle'      : 0.8,
		'knife'        : 0.7,
		'blunt_object' : 0.4,
		'replica'      : 0.3,
		'other'        : 0.2,
		'unarmed'      : 0.1,
		'not_available': 0,
	}
	armed_threat_score = [
		weapon_threat_ratio.get(threat_type, 0) for threat_type in
		threat_type_str.split(';')]
	return max(armed_threat_score) * 0.6


def scale_threat_type_score(threat_type):
	"""
    This function takes a string containing threat types
    and returns the danger score based on the 'threat_type_ratio' dictionary.
    Args:
        threat_type: String containing threat types (e.g., 'shoot')
    Returns:
        int: Highest danger score found in the string.
    """

	threat_type_ratio = {
		'shoot'        : 1,
		'attack'       : 0.8,
		'move'         : 0.6,
		'threat'       : 0.5,
		'point'        : 0.2,
		'flee'         : 0.1,
		'accident'     : 0.1,
		'not_available': 0
	}
	return threat_type_ratio.get(threat_type, 0) * 0.3


def scale_flee_status_score(flee_status):
	"""
    This function takes a string containing flee types
    and returns the danger score based on the 'flee_threat_ratio' dictionary.
    Args:
        flee_status: String containing threat types (e.g., 'car')
    Returns:
        int: Highest danger score found in the string.
    """

	flee_threat_ratio = {
		'car'          : 1,
		'foot'         : 0.5,
		'other'        : 0.1,
		'not'          : 0,
		'not_available': 0
	}
	return flee_threat_ratio.get(flee_status, 0) * 0.1


def get_threat_severity(threat_level):
	"""
    Determines the threat severity level based on a numerical threat score.
    Args:
        threat_level (float): A numerical threat score, typically between 0.0 and 1.0.
    Returns:
        str: A string indicating the threat severity level 
             ('Low Threat', 'Moderate Threat', 'Serious Threat',
              'Elevated Threat', or 'Extreme Threat').
	"""

	if threat_level <= 0.1:
		return 'Low Threat'
	elif threat_level <= 0.4:
		return 'Moderate Threat'
	elif threat_level <= 0.7:
		return 'Serious Threat'
	elif threat_level < 0.9:
		return 'Elevated Threat'
	else:
		return 'Extreme Threat'


df['threat_level'] = (
		df['armed_with'].apply(scale_armed_threat_score) +
		df['threat_type'].apply(scale_threat_type_score) +
		df['flee_status'].apply(scale_flee_status_score))

df['threat_level'] = df['threat_level'].apply(
	get_threat_severity).astype('category')

### Hunting for Duplicates

Let's make sure we're not double-counting incidents. Our first step: Identifying any fully duplicated rows in our dataset. This will be crucial for accurate analysis!


In [13]:
duplicated_rows = df[df.duplicated()]

if duplicated_rows.empty:
	print("There are no duplicated rows")
else:
	print(duplicated_rows)

There are no duplicated rows


In [14]:
df['namesake_count'] = (
	df.groupby('name')['name'].transform('count'))

repeating_name = df[
	(df['namesake_count'] > 1) &
	(df['name'] != 'not_available') &
	(df.duplicated(subset=['name', 'date'], keep=False))]

repeating_name = (
	repeating_name.sort_values(
		by=['namesake_count', 'name'], ascending=[False, True]))

repeating_name = repeating_name.loc[:, ['name', 'age', 'gender',
                                        'race', 'date', 'namesake_count']]
repeating_name

,name,age,gender,race,date,namesake_count
id,,,,,,
3692,Kelly G. Abbott,51.0,male,White,2018-05-11,2
3746,Kelly G. Abbott,51.0,male,White,2018-05-11,2
8642,Thomas Phan,40.0,male,Asian heritage,2022-11-16,2
8661,Thomas Phan,40.0,male,Asian heritage,2022-11-16,2


### Investigating Suspicious Duplicates

As I suspected... it seems highly unlikely that two incidents would share *all* those identical details. Let's dig deeper and see if there's a data entry error at play. 


In [15]:
df.loc[[3692, 3746, 8642, 8661]]

,date,threat_type,flee_status,armed_with,city,county,state,latitude,longitude,location_precision,...,race,race_source,was_mental_illness_related,body_camera,substituted_location,substituted_age,location_fatality_counter,age_group,threat_level,namesake_count
id,,,,,,,,,,,,,,,,,,,,,
3692,2018-05-11,shoot,car,gun,York,not_available,WI,44.595701,-90.476763,not_available,...,White,public_record,False,False,False,False,13,adult,Extreme Threat,2
3746,2018-05-11,shoot,car,gun,Clark County,not_available,WI,0.000000,0.000000,not_available,...,White,not_available,False,False,True,False,0,adult,Extreme Threat,2
8642,2022-11-16,attack,not,knife,Santa Clarita,not_available,CA,34.427815,-118.557983,not_available,...,Asian heritage,public_record,False,False,False,False,0,adult,Serious Threat,2
8661,2022-11-16,not_available,not_available,gun,Santa Ana,not_available,CA,33.745412,-117.850271,not_available,...,Asian heritage,clip,False,False,False,False,0,adult,Serious Threat,2


### Addressing Duplicate Names and Missing Data

Since these duplicate names seem suspicious, let's prioritize data integrity.  We'll remove entries with the following issues:

* Missing **location** features
* Missing **threat_type** features
* Missing **flee_status** features

This should help clean up potentially unreliable data points. 


In [16]:
df = df.drop(index=[3746, 8661])

### Data Formatting & Cleanup

Let's not forget to polish up those messy features!  We'll focus on:

* **Incorrect Feature Formats:** Ensuring data types are consistent for accurate analysis.  

In [17]:
df['date'] = pd.to_datetime(df['date'])
df['name'] = df['name'].astype('string')

categorical_columns = [
	'threat_type', 'flee_status', 'armed_with', 'city',
	'county', 'state', 'location_precision', 'gender', 'race',
	'race_source', 'namesake_count']

df[categorical_columns] = (
	df)[categorical_columns].astype('category')

### Extracting Detailed Datetime Formats

Let's unlock more time-based insights! We'll extract additional datetime components (**year**, **month**, **day of week**, **day**, etc.) for granular analysis later on.  


In [18]:
df['year'] = df['date'].dt.year.astype('category')
df['month'] = df['date'].dt.month_name().astype('category')
df['day_of_week'] = df['date'].dt.day_name().astype('category')
df['day'] = df['date'].dt.day.astype('category')

### Mastering the Seasons 

Why stop at basic dates? Let's write a function to determine the season (**Spring**, **Summer**, **Fall**, **Winter**) for each incident.  A touch of meteorological flair for our analysis! 


In [19]:
def categorize_season(date):
	"""Maps a date to its corresponding season.
    Args:
        date (datetime.date): A date object.
    Returns:
        str: The season corresponding to the date 
        ('Spring', 'Summer', 'Fall', or 'Winter').
    """

	if date.month in [3, 4, 5]:
		return 'Spring'
	elif date.month in [6, 7, 8]:
		return 'Summer'
	elif date.month in [9, 10, 11]:
		return 'Fall'
	else:
		return 'Winter'


df['season'] = (
	df['date'].apply(categorize_season).astype('category'))

### Investigating the Holiday Factor

Could holidays play a role in these incidents? Let's flag whether each incident occurred on a US federal holiday. This might reveal interesting patterns.


In [20]:
us_holidays = holidays.US()


def is_bank_holiday(date):
	"""
    Determines if a given date is a US bank holiday.
    Args:
        date (datetime.date): The date to check.
    Returns:
        bool: True if the date is a US bank holiday, False otherwise.
    """

	return date in us_holidays


df['bank_holiday'] = (
	df['date'].apply(is_bank_holiday).astype(bool))

### Lunar Investigations: Does the Moon Matter?

The moon's influence is a source of endless fascination.  Let's calculate the lunar phase (**full moon**, **new moon**, etc.) for each incident.  Who knows, we might uncover a hidden lunar correlation!


In [21]:
def calculate_lunar_phase(date):
	"""
    Determines the lunar phase for a given date.
    Uses the 'ephem' library for astronomical calculations.
    Args:
        date (datetime.date): The date for which to calculate the lunar phase.
    Returns:
        str: The lunar phase category 
        ('New Moon', 'First Quarter', 'Full Moon', or 'Last Quarter').
    """

	obs = ephem.Observer()
	obs.date = date
	moon_phase = ephem.Moon(obs).phase
	if moon_phase < 7.4:
		return 'New Moon'
	elif moon_phase < 14.8:
		return 'First Quarter'
	elif moon_phase < 22.1:
		return 'Full Moon'
	elif moon_phase < 29.5:
		return 'Last Quarter'
	else:
		return 'New Moon'


df['lunar_phase'] = (
	df['date'].apply(calculate_lunar_phase).astype('category'))

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9495 entries, 3 to 10272
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        9495 non-null   datetime64[ns]
 1   threat_type                 9495 non-null   category      
 2   flee_status                 9495 non-null   category      
 3   armed_with                  9495 non-null   category      
 4   city                        9495 non-null   category      
 5   county                      9495 non-null   category      
 6   state                       9495 non-null   category      
 7   latitude                    9495 non-null   float64       
 8   longitude                   9495 non-null   float64       
 9   location_precision          9495 non-null   category      
 10  name                        9495 non-null   string        
 11  age                         9495 non-null   float64       
 

### Final Touches: Saving Our Polished Dataframe

Our data is gleaming! Let's preserve this beautifully formatted dataset by saving it in Parquet format.  This ensures consistency and efficiency for future exploration. 


In [23]:
df.to_parquet('police_brutality.parquet')